In [1]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

In [2]:
with open('all_malware.txt', 'r', encoding='utf-8') as f:
    text = f.read()
malware = []

for line in text.split('\n')[:-1]:
    malware.append(line)

In [3]:
len(malware)

1300

In [4]:
with open('all_threat_actors.txt', 'r', encoding='utf-8') as f:
    text = f.read()
actors = []

for line in text.split('\n')[:-1]:
    actors.append(line)

In [5]:
len(actors)

235

In [6]:
def read_triples(fname):
    triples = []
    with open(fname, 'r', encoding='utf-8') as f:
        text = f.read()

    for line in text.split('\n'):
        if len(line) > 0:
            e1, r, e2 = line.split('\t')
            triples.append([e1, r, e2])
    return triples

In [7]:
triples = read_triples('150_all.txt')

In [8]:
def get_malware_nodes(mal, triples):
    nodes = set()
    
    for e1, r, e2 in triples:
        if e1 == mal:
            nodes.add((e2, r))
        elif e2 == mal:
            nodes.add((e1, r))
    return nodes

In [9]:
def get_all_malware_nodes(triples):
    malware_nodes = {}
    for m in malware:
        nodes = get_malware_nodes(m, triples)
        if len(nodes) > 0:
            malware_nodes[m] = nodes
    return malware_nodes

In [10]:
def get_all_actor_nodes(triples):
    actor_nodes = {}
    for m in actors:
        nodes = get_malware_nodes(m, triples)
        nodes_list = list(nodes)
        for x in nodes_list:
            if x[1] == 'hasAuthor':
                mal_nodes = get_malware_nodes(x[0], triples)
                for z in mal_nodes:
                    if z[1] in ['targets', 'uses', 'exploits', 'indicates', 'isA', 'variantOf',]:
                        nodes.add(z)
#                     else:
#                         nodes.add(x[0], )
        if len(nodes) > 0:
            actor_nodes[m] = nodes
    return actor_nodes

In [11]:
malware_nodes = get_all_malware_nodes(triples)

In [12]:
# malware_nodes

In [13]:
def get_distance(node1, node2, type='jaccard'):
    union = node1.union(node2)
    intersect = node1.intersection(node2)
    if type == 'intersect':
        return 1000-len(intersect)
    elif type == 'jaccard':
        return 1 - len(intersect)/len(union)
    elif type == 'overlap':
        return 1 - len(intersect)/min(len(node1), len(node2))

In [14]:
# from sklearn.manifold import TSNE

In [15]:
# def cluster(triple_fname, topn=-1):
#     triples = read_triples(triple_fname)
#     malware_nodes = get_all_malware_nodes(triples)

#     malware_list = list(malware_nodes.keys())
    
#     if topn > 0:
#         malware_node_count = []
#         for k, v in malware_nodes.items():
#             malware_node_count.append([k, len(v)])
#         malware_node_count.sort(key=lambda x: x[1], reverse=True)
#         malware_list = [x[0] for x in malware_node_count[:topn]]

        

#     dist = []

#     for i in range(len(malware_list)):
#         di = []
#         mal_node_i = malware_nodes[malware_list[i]]
#         for j in range(len(malware_list)):
#             mal_node_j = malware_nodes[malware_list[j]]
#     #             print(malware_list[i], malware_list[j])
#     #             print(mal_node_i)
#     #             print(mal_node_j)
#     #             print(get_distance(mal_node_i, mal_node_j))
#     #             print('='*100)
#             di.append(get_distance(mal_node_i, mal_node_j, 'jaccard'))
#         dist.append(di)
#     #     print(dist)
    
#     db = DBSCAN(metric='precomputed', eps=.85, min_samples=2)
#     db.fit(dist)
    
#     core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
#     core_samples_mask[db.core_sample_indices_] = True
#     labels = db.labels_
    
#     n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
#     n_noise_ = list(labels).count(-1)
    
#     X = TSNE(n_components=2, metric='precomputed').fit_transform(dist) 
    
#     unique_labels = set(labels)
#     colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
#     for k, col in zip(unique_labels, colors):
#         if k == -1:
#             continue
#             #Black used for noise.
#             col = [0, 0, 0, 1]

#         class_member_mask = labels == k

#         xy = X[class_member_mask & core_samples_mask]
#         plt.plot(
#             xy[:, 0],
#             xy[:, 1],
#             "o",
#             markerfacecolor=tuple(col),
#             markeredgecolor="k",
#             markersize=14,
#         )

#         xy = X[class_member_mask & ~core_samples_mask]
#         plt.plot(
#             xy[:, 0],
#             xy[:, 1],
#             "o",
#             markerfacecolor=tuple(col),
#             markeredgecolor="k",
#             markersize=6,
#         )

#     plt.title("Estimated number of clusters: %d" % n_clusters_)
#     plt.show()

In [16]:
# cluster('150_all.txt', -1)

In [17]:
# cluster('12k_all.txt', -1)

In [18]:
# cluster('12k_all.txt', 100)

In [315]:
# triples = read_triples('150_all.txt')
# malware_nodes = get_all_malware_nodes(triples)

# malware_list = list(malware_nodes.keys())

# dist = []

# for i in range(len(malware_list)):
#     di = []
#     mal_node_i = malware_nodes[malware_list[i]]
#     for j in range(len(malware_list)):
#         mal_node_j = malware_nodes[malware_list[j]]
# #             print(malware_list[i], malware_list[j])
# #             print(mal_node_i)
# #             print(mal_node_j)
# #             print(get_distance(mal_node_i, mal_node_j))
# #             print('='*100)
#         di.append(get_distance(mal_node_i, mal_node_j))
#     dist.append(di)
# #     print(dist)

In [316]:
# db = DBSCAN(metric='precomputed', eps=0.2, min_samples=2)
# db.fit(dist)

In [317]:
# core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
# core_samples_mask[db.core_sample_indices_] = True
# labels = db.labels_

In [318]:
# n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
# n_noise_ = list(labels).count(-1)

In [319]:
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA

In [320]:
# X = PCA(n_components=2).fit_transform(dist) 

In [321]:
# # Black removed and is used for noise instead.
# unique_labels = set(labels)
# colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
# for k, col in zip(unique_labels, colors):
#     if k == -1:
#         # Black used for noise.
#         col = [0, 0, 0, 1]

#     class_member_mask = labels == k

#     xy = X[class_member_mask & core_samples_mask]
#     plt.plot(
#         xy[:, 0],
#         xy[:, 1],
#         "o",
#         markerfacecolor=tuple(col),
#         markeredgecolor="k",
#         markersize=14,
#     )

#     xy = X[class_member_mask & ~core_samples_mask]
#     plt.plot(
#         xy[:, 0],
#         xy[:, 1],
#         "o",
#         markerfacecolor=tuple(col),
#         markeredgecolor="k",
#         markersize=6,
#     )

# plt.title("Estimated number of clusters: %d" % n_clusters_)
# plt.show()

In [23]:
def find_most_similar_malware(mal, triple_fname):
    triples = read_triples(triple_fname)
    malware_nodes = get_all_malware_nodes(triples)

    malware_list = list(malware_nodes.keys())
    mal_node_i = malware_nodes[mal]
        
    dist = []    
    for j in range(len(malware_list)):
            mal_node_j = malware_nodes[malware_list[j]]
            dist.append([malware_list[j], get_distance(mal_node_i, mal_node_j, 'jaccard')])
    dist.sort(key=lambda x: x[1])
    
    return dist[1:6]

In [20]:
find_most_similar_malware('FluBot', '12k_all.txt')

{('T1474', 'uses'), ('T1640', 'uses'), ('AVG', 'targets'), ('T1616', 'uses'), ('Amazon', 'targets'), ('getty', 'targets'), ('UPS', 'targets'), ('Sparkasse', 'targets'), ('T1418', 'uses'), ('German', 'targets'), ('2019.', 'discoveredIn'), ('June', 'discoveredIn'), ('T1582', 'uses'), ('Argos', 'targets'), ('Proofpoint', 'targets'), ('T1409', 'uses'), ('ransomware', 'isA'), ('WhatsApp', 'targets'), ('020870ff50253429fb2aa56cd948c18329fe1ad3', 'indicates'), ('Ireland', 'targets'), ('T1456', 'uses'), ('T1625', 'uses'), ('Barcelona', 'has'), ('T1429', 'uses'), ('eb41401c48e8c336348f50a1ec6d90e75ae60460', 'indicates'), ('ZDNet', 'targets'), ('2018.', 'discoveredIn'), ('29.09.2021', 'discoveredIn'), ('Vodafone', 'targets'), ('Telegram', 'targets'), ('apps', 'targets'), ('Morrisons', 'targets'), ('Australian', 'targets'), ('access', 'targets'), ('T1639', 'uses'), ('Botnet', 'isA'), ('Chrome', 'targets'), ('malware', 'isA'), ('T1604', 'uses'), ('Netcraft', 'targets'), ('trojan', 'isA'), ('Correo

[['TeaBot', 0.7906976744186046],
 ['Medusa', 0.8064516129032258],
 ['Gustuff', 0.8115942028985508],
 ['Ghimob', 0.823943661971831],
 ['Faketoken', 0.8260869565217391]]

In [21]:
triples = read_triples('12k_all.txt')
malware_nodes = get_all_malware_nodes(triples)

In [22]:
mal_node_i = malware_nodes['FluBot']
mal_node_j = malware_nodes['TeaBot']

print(mal_node_i.intersection(mal_node_j))

{('T1481', 'uses'), ('T1616', 'uses'), ('T1640', 'uses'), ('T1635', 'uses'), ('T1512', 'uses'), ('apps', 'targets'), ('UPS', 'targets'), ('T1639', 'uses'), ('Android', 'targets'), ('German', 'targets'), ('T1626', 'uses'), ('T1582', 'uses'), ('T1513', 'uses'), ('Google', 'targets'), ('T1409', 'uses'), ('Spanish', 'targets'), ('ransomware', 'isA'), ('WhatsApp', 'targets'), ('trojan', 'isA'), ('Correos', 'targets'), ('ESET', 'targets'), ('T1636', 'uses'), ('T1625', 'uses'), ('bank', 'targets'), ('T1629', 'uses'), ('T1406', 'uses'), ('T1418', 'uses')}


In [24]:
def find_most_similar_threat_actor(act, triple_fname):
    triples = read_triples(triple_fname)
    actor_nodes = get_all_actor_nodes(triples)
    
    actor_list = list(actor_nodes.keys())
    actor_node_i = actor_nodes[act]
    
    dist = []    
    for j in range(len(actor_list)):
            actor_node_j = actor_nodes[actor_list[j]]
            dist.append([actor_list[j], get_distance(actor_node_i, actor_node_j, 'jaccard')])
    dist.sort(key=lambda x: x[1])
    
    return dist[1:6]

In [26]:
find_most_similar_threat_actor('APT15', '12k_all.txt')

[['GREF', 0.5333333333333333],
 ['Boyusec', 0.574468085106383],
 ['Ke3chang', 0.5833333333333333],
 ['APT-C-50', 0.8163265306122449],
 ['Kitten', 0.8333333333333334]]

In [27]:
triples = read_triples('12k_all.txt')
actor_nodes = get_all_actor_nodes(triples)

In [28]:
mal_node_i = actor_nodes['APT15']
mal_node_j = actor_nodes['Boyusec']

print(mal_node_i.intersection(mal_node_j))

{('TIBBIYJAWHAR', 'targets'), ('Telegram', 'targets'), ('Kuwait', 'targets'), ('apps', 'targets'), ('third-party', 'targets'), ('Spyware', 'isA'), ('TalkBox', 'targets'), ('surveillanceware', 'isA'), ('Uyghur', 'targets'), ('Android', 'targets'), ('Voxer', 'targets'), ('Google', 'targets'), ('Syria', 'targets'), ('Xinjiang', 'targets'), ('China', 'targets'), ('music', 'targets'), ('Central', 'targets'), ('RAT', 'isA'), ('Muslim', 'targets'), ('voice', 'targets')}
